# Deposit Fails Due to Min-ADA Requirement

This test case illustrates that Marlowe contracts involving native tokens may contain unexecutable `Deposit` actions due to the [min-ADA](https://mantis.functionally.io/how-to/min-ada-value/) requirement that a certain amount of ADA always reside in a UTxO containing a native token. Under the protocol parameters during the Alonzo Era, the min-ADA varies from approximately 1.3 ADA to 22.5 ADA. Although this requirement may block certain paths through an otherwise valid Marlowe contract, it does not cause funds to be locked forever: rather, there will always be an executable path through the contract that leads to its closing and the dispersement of funds from the contract.

👉 This example highlights the importance of performing static analysis of a Marlowe contract *before* creating the contract on the blockchain. To verify that a contract will not experience this failure mode, one needs to compute the min-ADA requirement (via a tool like `cardano-cli transaction calculate-min-required-utxo`) at *every `Deposit` and `Pay`* in the contract to verify that the ADA balance exceeds min-ADA at each of those locations.

## Preliminaries

Select the network.

In [1]:
CARDANO_TESTNET_MAGIC=1567
CARDANO_NODE_SOCKET_PATH=node.socket

Select the payment credentials.

In [2]:
PAYMENT_SKEY=payment.skey
PAYMENT_VKEY=payment.vkey
PAYMENT_HASH=$(cardano-cli address key-hash --payment-verification-key-file $PAYMENT_VKEY)
echo $PAYMENT_HASH
PAYMENT_ADDR=$(cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file $PAYMENT_VKEY)
echo $PAYMENT_ADDR

0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07
addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


Set the token name.

In [3]:
TOKEN_NAME="Token requiring min 1.448244 ADA"

Mint the token.

In [4]:
POLICY_ID=$(
marlowe-cli util mint --required-signer $PAYMENT_SKEY \
                      --change-address $PAYMENT_ADDR  \
                      --out-file /dev/null            \
                      --submit 600                    \
                      "$TOKEN_NAME"                   \
| sed -e 's/PolicyID "\(.*\)"$/\1/'                   \
)
echo $POLICY_ID

8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d


View the result of the minting.

In [5]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $PAYMENT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
6a41f880e754c8e692bb02008e8906b21119ba18156ccb078cb95e650995b811     0        2303800612582 lovelace + TxOutDatumNone
6a41f880e754c8e692bb02008e8906b21119ba18156ccb078cb95e650995b811     1        10000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.546f6b656e20726571756972696e67206d696e20312e34343832343420414441 + TxOutDatumNone


## *Control Case:* Successful Transaction with Sufficient Min-ADA

Select the UTxOs for later use in the transactions.

In [6]:
TX_0_ADA=$(marlowe-cli util select --lovelace-only 10000000 $PAYMENT_ADDR | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | head)
echo $TX_0_ADA

6a41f880e754c8e692bb02008e8906b21119ba18156ccb078cb95e650995b811#0


In [7]:
TX_0_TOKEN=$(marlowe-cli util select --asset-only "$POLICY_ID.$TOKEN_NAME" $PAYMENT_ADDR | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/')
echo $TX_0_TOKEN

6a41f880e754c8e692bb02008e8906b21119ba18156ccb078cb95e650995b811#1


### Design the contract and its initial state

Find the current time.

In [8]:
NOW=$(($(date -u +%s) * 1000))
MINUTE=$((60 * 1000))
echo $NOW

1658968791000


Create a contract which receives a deposit of the token, waits for a notification, and then pays the token.

In [9]:
cat > min-ada.contract << EOI
{
  "when" : [
    {
      "case" : {
        "party" : {"pk_hash" : "$PAYMENT_HASH"}
      , "deposits" : 1
      , "of_token" : {
          "currency_symbol" : "$POLICY_ID"
        , "token_name" : "$TOKEN_NAME"
        }
      , "into_account" : {"pk_hash" : "$PAYMENT_HASH"}
      }
     , "then" : {
         "when" : [
           {
             "case" :{ 
               "notify_if" : true
              }
           , "then":"close"
           }
         ]
       , "timeout_continuation" : "close"
       , "timeout" : $((NOW + 5 * MINUTE))
       }
     }
   ]
, "timeout_continuation" : "close"
, "timeout" : $((NOW + 5 * MINUTE))
}
EOI
json2yaml min-ada.contract

timeout: 1658969091000
timeout_continuation: close
when:
- case:
    deposits: 1
    into_account:
      pk_hash: 0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07
    of_token:
      currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: Token requiring min 1.448244 ADA
    party:
      pk_hash: 0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07
  then:
    timeout: 1658969091000
    timeout_continuation: close
    when:
    - case:
        notify_if: true
      then: close


The contract will start with 2 ADA, which is sufficient to cover the min-ADA requirement throughout the life of the contract.

In [10]:
INITIAL_LOVELACE=2000000

Create the initial state of the contract, which starts with the initial ADA.

In [11]:
cat > min-ada.state << EOI
{
  "choices": [],
  "accounts": [
    [
      [
        {
          "pk_hash": "$PAYMENT_HASH"
        },
        {
          "currency_symbol": "",
          "token_name": ""
        }
      ],
      $INITIAL_LOVELACE
    ]
  ],
  "minTime": $NOW,
  "boundValues": []
}
EOI
json2yaml min-ada.state

accounts:
- - - pk_hash: 0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1658968791000


### 1. Create the contract

Create the Plutus script and the initial datum.

In [12]:
marlowe-cli run initialize --contract-file min-ada.contract \
                           --state-file    min-ada.state    \
                           --out-file      min-ada-1.marlowe

Compute the address of the Marlowe script.

In [13]:
SCRIPT_ADDR=$(jq -r .marloweValidator.address min-ada-1.marlowe)
echo $SCRIPT_ADDR

addr_test1wp3v2mx0ccsh4l6kjtsa86lgnsss20f3lsgcst9jr07axpcmajgcl


Submit the transaction to create the contract.

In [14]:
TX_1=$(
marlowe-cli run execute --required-signer $PAYMENT_SKEY      \
                        --tx-in $TX_0_ADA                    \
                        --marlowe-out-file min-ada-1.marlowe \
                        --change-address $PAYMENT_ADDR       \
                        --out-file /dev/null                 \
                        --submit 600                         \
                        --print-stats                        \
| sed -e 's/^TxId "\(.*\)"$/\1/'                             \
)
echo "TxId \"$TX_1\""


Fee: Lovelace 188249
Size: 504 / 32768 = 1%
Execution units:
  Memory: 0 / 30000000 = 0%
  Steps: 0 / 10000000000 = 0%
TxId "83562a16c19d0c41de032a6568b51d8ba2a1788f31ded2daf15d0c2530958d5d"


The script address now has the contract with its initial state.

In [15]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
83562a16c19d0c41de032a6568b51d8ba2a1788f31ded2daf15d0c2530958d5d     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "dad1cf3a390d2953a1019c209350813572fc5b2787ddfb4afe431baee35c8184"


### 2. Deposit the token

Prepare the transaction for depositing the token into the contract.

In [16]:
marlowe-cli run prepare --marlowe-file min-ada-1.marlowe          \
                        --deposit-account PK=$PAYMENT_HASH        \
                        --deposit-party PK=$PAYMENT_HASH          \
                        --deposit-token "$POLICY_ID.$TOKEN_NAME"  \
                        --deposit-amount 1                        \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $((NOW + 4 * MINUTE)) \
                        --out-file min-ada-2.marlowe

Submit the transaction to deposit the token.

In [17]:
TX_2=$(
marlowe-cli run execute --required-signer $PAYMENT_SKEY      \
                        --tx-in "$TX_0_TOKEN"                \
                        --tx-in "$TX_1#0"                    \
                        --tx-in-collateral "$TX_1#0"         \
                        --tx-in-marlowe "$TX_1#1"            \
                        --marlowe-in-file  min-ada-1.marlowe \
                        --marlowe-out-file min-ada-2.marlowe \
                        --change-address $PAYMENT_ADDR       \
                        --out-file /dev/null                 \
                        --submit 600                         \
                        --print-stats                        \
| sed -e 's/^TxId "\(.*\)"$/\1/'                             \
)
echo "TxId \"$TX_2\""


Fee: Lovelace 1123677
Size: 13649 / 32768 = 41%
Execution units:
  Memory: 4219780 / 30000000 = 14%
  Steps: 1450621095 / 10000000000 = 14%
TxId "e351668cbc0ed1f4168604b778a02879481cc12198bfc3eabd1cc4a9051111e6"


View the new UTxO at the script address.

In [18]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
e351668cbc0ed1f4168604b778a02879481cc12198bfc3eabd1cc4a9051111e6     1        2000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.546f6b656e20726571756972696e67206d696e20312e34343832343420414441 + TxOutDatumHash ScriptDataInAlonzoEra "397e99881a94613502057eb525cd0b6f3e0b5a096cb4da4e5b2d214bb2dbe563"


### 3. Notify the contract to close

Prepare the notification.

In [19]:
marlowe-cli run prepare --marlowe-file min-ada-2.marlowe          \
                        --notify                                  \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $((NOW + 4 * MINUTE)) \
                        --out-file min-ada-3.marlowe

Payment 1
  Acccount: PK "0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07"
  Payee: Party (PK "0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07")
  Ada: 2.000000
Payment 2
  Acccount: PK "0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07"
  Payee: Party (PK "0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07")
  Ada: 0.000000
  8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d."Token requiring min 1.448244 ADA": 1


Submit the notification.

In [20]:
TX_3=$(
marlowe-cli run execute --required-signer $PAYMENT_SKEY      \
                        --tx-in "$TX_2#0"                    \
                        --tx-in-collateral "$TX_2#0"         \
                        --tx-in-marlowe "$TX_2#1"            \
                        --marlowe-in-file  min-ada-2.marlowe \
                        --marlowe-out-file min-ada-3.marlowe \
                        --change-address $PAYMENT_ADDR       \
                        --out-file /dev/null                 \
                        --submit 600                         \
                        --print-stats                        \
| sed -e 's/^TxId "\(.*\)"$/\1/'                             \
)
echo "TxId \"$TX_3\""


Fee: Lovelace 1030934
Size: 13172 / 32768 = 40%
Execution units:
  Memory: 3421362 / 30000000 = 11%
  Steps: 1156000626 / 10000000000 = 11%
TxId "3e9ca0c50476c0b82ce0cd1a7f6837dc36af5c2401a1f84be0fc4fb56d0b032b"


Since the contract has closed, there is no UTxO at the script address.

In [21]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


The initial lovelace and the token have been paid back.

In [22]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $PAYMENT_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3e9ca0c50476c0b82ce0cd1a7f6837dc36af5c2401a1f84be0fc4fb56d0b032b     0        2303806269722 lovelace + TxOutDatumNone
3e9ca0c50476c0b82ce0cd1a7f6837dc36af5c2401a1f84be0fc4fb56d0b032b     1        2000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.546f6b656e20726571756972696e67206d696e20312e34343832343420414441 + TxOutDatumNone


## *Failure Case:* Unable to Deposit Token

Select the UTxOs for later use in the transactions.

In [23]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $PAYMENT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3e9ca0c50476c0b82ce0cd1a7f6837dc36af5c2401a1f84be0fc4fb56d0b032b     0        2303806269722 lovelace + TxOutDatumNone
3e9ca0c50476c0b82ce0cd1a7f6837dc36af5c2401a1f84be0fc4fb56d0b032b     1        2000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.546f6b656e20726571756972696e67206d696e20312e34343832343420414441 + TxOutDatumNone


In [24]:
TX_0_ADA=$(marlowe-cli util select --lovelace-only 10000000 $PAYMENT_ADDR | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | head)
echo $TX_0_ADA

3e9ca0c50476c0b82ce0cd1a7f6837dc36af5c2401a1f84be0fc4fb56d0b032b#0


In [25]:
TX_0_TOKEN=$(marlowe-cli util select --asset-only "$POLICY_ID.$TOKEN_NAME" $PAYMENT_ADDR | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/')
echo $TX_0_TOKEN

3e9ca0c50476c0b82ce0cd1a7f6837dc36af5c2401a1f84be0fc4fb56d0b032b#1


### Design the contract

In [26]:
NOW=$(($(date -u +%s) * 1000))
MINUTE=$((60 * 1000))
echo $NOW

1658968930000


Create a contract which receives a deposit of the token, waits for a notification, and then pays the token.

In [27]:
cat > min-ada.contract << EOI
{
  "when" : [
    {
      "case" : {
        "party" : {"pk_hash" : "$PAYMENT_HASH"}
      , "deposits" : 1
      , "of_token" : {
          "currency_symbol" : "$POLICY_ID"
        , "token_name" : "$TOKEN_NAME"
        }
      , "into_account" : {"pk_hash" : "$PAYMENT_HASH"}
      }
     , "then" : {
         "when" : [
           {
             "case" :{ 
               "notify_if" : true
              }
           , "then":"close"
           }
         ]
       , "timeout_continuation" : "close"
       , "timeout" : $((NOW + 5 * MINUTE))
       }
     }
   ]
, "timeout_continuation" : "close"
, "timeout" : $((NOW + 5 * MINUTE))
}
EOI
json2yaml min-ada.contract

timeout: 1658969230000
timeout_continuation: close
when:
- case:
    deposits: 1
    into_account:
      pk_hash: 0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07
    of_token:
      currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: Token requiring min 1.448244 ADA
    party:
      pk_hash: 0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07
  then:
    timeout: 1658969230000
    timeout_continuation: close
    when:
    - case:
        notify_if: true
      then: close


The contract will start with 1.4 ADA, which is **not** sufficient to cover the min-ADA requirement throughout the life of the contract.

In [28]:
INITIAL_LOVELACE=1400000

Create the initial state of the contract, which starts with the initial ADA.

In [29]:
cat > min-ada.state << EOI
{
  "choices": [],
  "accounts": [
    [
      [
        {
          "pk_hash": "$PAYMENT_HASH"
        },
        {
          "currency_symbol": "",
          "token_name": ""
        }
      ],
      $INITIAL_LOVELACE
    ]
  ],
  "minTime": $NOW,
  "boundValues": []
}
EOI
json2yaml min-ada.state

accounts:
- - - pk_hash: 0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07
    - currency_symbol: ''
      token_name: ''
  - 1400000
boundValues: []
choices: []
minTime: 1658968930000


### 1. Create the contract

Create the Plutus script and the initial datum.

In [30]:
marlowe-cli run initialize --contract-file min-ada.contract \
                           --state-file    min-ada.state    \
                           --out-file      min-ada-1.marlowe

Compute the address of the Marlowe script.

In [31]:
SCRIPT_ADDR=$(jq -r .marloweValidator.address min-ada-1.marlowe)
echo $SCRIPT_ADDR

addr_test1wp3v2mx0ccsh4l6kjtsa86lgnsss20f3lsgcst9jr07axpcmajgcl


Submit the transaction to create the contract.

In [32]:
TX_1=$(
marlowe-cli run execute --required-signer $PAYMENT_SKEY      \
                        --tx-in $TX_0_ADA                    \
                        --marlowe-out-file min-ada-1.marlowe \
                        --change-address $PAYMENT_ADDR       \
                        --out-file /dev/null                 \
                        --submit 600                         \
                        --print-stats                        \
| sed -e 's/^TxId "\(.*\)"$/\1/'                             \
)
echo "TxId \"$TX_1\""


Fee: Lovelace 188249
Size: 504 / 32768 = 1%
Execution units:
  Memory: 0 / 30000000 = 0%
  Steps: 0 / 10000000000 = 0%
TxId "cd60f7caf14f105d7fc4de03417119619ca412adb87c650b60315918381c79c0"


The script address now has the contract with its initial state.

In [33]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
cd60f7caf14f105d7fc4de03417119619ca412adb87c650b60315918381c79c0     1        1400000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "fbc34ba877f12f17881dd2296d365ca34dffa1516f2ed7037c9af14f69f107f6"


### Attempt to deposit the token, but fail

Prepare the transaction for depositing the token into the contract.

In [34]:
marlowe-cli run prepare --marlowe-file min-ada-1.marlowe          \
                        --deposit-account PK=$PAYMENT_HASH        \
                        --deposit-party PK=$PAYMENT_HASH          \
                        --deposit-token "$POLICY_ID.$TOKEN_NAME"  \
                        --deposit-amount 1                        \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $((NOW + 4 * MINUTE)) \
                        --out-file min-ada-2.marlowe

Submit the transaction to deposit the token.

In [35]:
marlowe-cli run execute --required-signer $PAYMENT_SKEY      \
                        --tx-in "$TX_0_TOKEN"                \
                        --tx-in "$TX_1#0"                    \
                        --tx-in-collateral "$TX_1#0"         \
                        --tx-in-marlowe "$TX_1#1"            \
                        --marlowe-in-file  min-ada-1.marlowe \
                        --marlowe-out-file min-ada-2.marlowe \
                        --change-address $PAYMENT_ADDR       \
                        --out-file /dev/null                 \
                        --submit 600                         \
                        --print-stats

TxBodyErrorMinUTxONotMet (TxOutInAnyEra AlonzoEra (TxOut (AddressInEra (ShelleyAddressInEra ShelleyBasedEraAlonzo) (ShelleyAddress Testnet (ScriptHashObj (ScriptHash "62c56ccfc6217aff5692e1d3ebe89c21053d31fc11882cb21bfdd307")) StakeRefNull)) (TxOutValue MultiAssetInAlonzoEra (valueFromList [(AdaAssetId,1400000),(AssetId "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d" "Token requiring min 1.448244 ADA",1)])) (TxOutDatum' ScriptDataInAlonzoEra "462bb67b7d140596aacac90caf95ecca69eb1fc9708aad49ea5468c6b16ee931" (ScriptDataConstructor 0 [ScriptDataConstructor 0 [ScriptDataMap [(ScriptDataConstructor 0 [ScriptDataConstructor 0 [ScriptDataBytes "\n\DC1\176\199\226]\197\217\198\&1q\189\243\157\151A\185\SOH\220\144>\DC2\180\225b4\142\a"],ScriptDataConstructor 0 [ScriptDataBytes "",ScriptDataBytes ""]],ScriptDataNumber 1400000),(ScriptDataConstructor 0 [ScriptDataConstructor 0 [ScriptDataBytes "\n\DC1\176\199\226]\197\217\198\&1q\189\243\157\151A\185\SOH\220\144>\DC2\180\225b4\142\a"]

: 1

The transaction failed with `TxBodyErrorMinUTxONotMet` because the script address only contains 1.4 ADA, but more ADA is needed to cover the min-ADA requirement. ***This `Case` statement in the contract can never be executed.***

### 2. Wait for a timeout and close the contract

Although the contract cannot proceed along its intended path with the deposit of the token, the contract is not blocked forever because the timeout for the `When` statement lets the contract close.

Prepare the transaction to close the contract.

In [36]:
marlowe-cli run prepare --marlowe-file min-ada-1.marlowe          \
                        --invalid-before $((NOW + 5 * MINUTE))    \
                        --invalid-hereafter $((NOW + 9 * MINUTE)) \
                        --out-file min-ada-2.marlowe

Payment 1
  Acccount: PK "0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07"
  Payee: Party (PK "0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07")
  Ada: 1.400000


Wait for the timeout, so that the contract can be closed.

In [37]:
sleep 5m

Submit the transaction to close the contract.

In [38]:
TX_2=$(
marlowe-cli run execute --required-signer $PAYMENT_SKEY      \
                        --tx-in "$TX_1#0"                    \
                        --tx-in-collateral "$TX_1#0"         \
                        --tx-in-marlowe "$TX_1#1"            \
                        --marlowe-in-file  min-ada-1.marlowe \
                        --marlowe-out-file min-ada-2.marlowe \
                        --change-address $PAYMENT_ADDR       \
                        --out-file /dev/null                 \
                        --submit 600                         \
                        --print-stats                        \
| sed -e 's/^TxId "\(.*\)"$/\1/'                             \
)
echo "TxId \"$TX_2\""


Fee: Lovelace 946434
Size: 13156 / 32768 = 40%
Execution units:
  Memory: 2358820 / 30000000 = 7%
  Steps: 844119786 / 10000000000 = 8%
TxId "a59695584207ad1ecad9f44cc4a98de5ad530c355cefcd365789b310322cfe5a"


Since the contract has closed, there is no UTxO at the script address.

In [39]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


The initial lovelace has been paid back.

In [40]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $PAYMENT_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a59695584207ad1ecad9f44cc4a98de5ad530c355cefcd365789b310322cfe5a     0        2303803735039 lovelace + TxOutDatumNone
a59695584207ad1ecad9f44cc4a98de5ad530c355cefcd365789b310322cfe5a     1        1400000 lovelace + TxOutDatumNone


## Clean Up

Burn the token.

In [41]:
marlowe-cli util mint --required-signer $PAYMENT_SKEY \
                      --change-address $PAYMENT_ADDR  \
                      --out-file /dev/null            \
                      --submit 600                    \
                      --count -1                      \
                      "$TOKEN_NAME"

PolicyID "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d"


In [42]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $PAYMENT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5e833cf44dead4d0eadb5ad31ba8e1e54b44c658531ff2210fc2e2e961aa7563     0        2303806952598 lovelace + TxOutDatumNone
